In [397]:
from xml.etree import ElementTree as ET
import folium
import numpy as np
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2
import ast

Lógica para a execução do código
1. listar todos os carros que rodaram no dia
2. listar as linhas que estes carros rodaram
3. listar as rotas ociosas de cada linha
4. listar as rotas programadas de cada linha
5. listar a rota realizada pelo carro (dados mix)
6. fazer o comparativo entre realizada e programada
7. enviar email com os carros ofensores

Todos os dados estão no banco postgres.

In [398]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'
port = '5440' 

In [399]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [400]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Paths

In [401]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

# SQL

In [402]:
sql_geodata = """ 
select a.execution_date, a.registration, a.description, a.coordinates
    from hp.raw_geodata a
"""

In [403]:
sql_idle = """ 
select a.id, a.num_linha, a.descricao, b.direcao, b.pontos_geolocalizacao
from
    hp.raw_linhas a,
    hp.raw_trajetos b
where a.id = b.linha_id
"""

In [404]:
sql_schedule = """ 
SELECT id, "data", linha, carro, re, nome, dthr_saida, dthr_retorno, dthr_chegada
FROM hp.raw_escalas_programadas rep
"""

In [405]:
sql_routes = """
SELECT id, linha_id, direcao, pontos_geolocalizacao
FROM hp.raw_trajetos;
"""

# Dataframes

In [406]:
df_trip_made = pd.read_sql(sql_geodata, engine)

In [407]:
df_trip_idle = pd.read_sql(sql_idle, engine)

In [408]:
df_schedule = pd.read_sql(sql_schedule, engine)

In [409]:
df_routes = pd.read_sql(sql_routes, engine)

# Functions

In [410]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [411]:
def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

In [412]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)

        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)

            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)

In [413]:
def haversine(lon1, lat1, lon2, lat2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Conversão de coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Diferenças nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [414]:
def extract_coords(coords_list):
    return [[coord['lat'], coord['lng']] for coord in coords_list]

# Transformações

# df_schedule

In [415]:
df_schedule.dtypes

id                       int64
data                    object
linha                   object
carro                    int64
re                      object
nome                    object
dthr_saida      datetime64[ns]
dthr_retorno    datetime64[ns]
dthr_chegada    datetime64[ns]
dtype: object

In [416]:
if df_schedule['data'].dtype == 'object':
    df_schedule['data'] = pd.to_datetime(df_schedule['data'])

## df_planned_trp

In [417]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [418]:
df_planned_trip = pd.DataFrame(placemarks_planned)

In [419]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])

In [420]:
df_planned_trip['identificacao'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[1])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [421]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])
df_planned_trip['Descrição e Sentido'] = df_planned_trip['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip['Descrição'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_planned_trip['Sentido'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip.drop('Descrição e Sentido', axis=1, inplace=True)

In [422]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [423]:
df_planned_trip['sublinha'] = df_planned_trip['identificacao'].apply(lambda x: de_para[x[:-1]])

In [424]:
df_planned_trip['num_linha'] = df_planned_trip['Código'].astype(str) + df_planned_trip['sublinha']

## df_trip_idle

In [425]:
df_trip_idle['num_linha_numerico'] = df_trip_idle['num_linha'].str.extract(r'^(\d+)')

In [426]:
df_trip_idle['direcao']=df_trip_idle['direcao'].str.upper()

## df_trip_made

In [427]:
df_trip_made.dtypes

execution_date    object
registration      object
description       object
coordinates       object
dtype: object

In [428]:
df_trip_made['execution_date'] = pd.to_datetime(df_trip_made['execution_date'])

In [429]:
df_trip_made['description'] = df_trip_made['description'].astype('Int64')

# Execution

* Lógica para a execução do código
* listar todos os carros que rodaram no dia
* listar as linhas que estes carros rodaram
* listar as rotas ociosas de cada linha
* listar as rotas programadas de cada linha
* listar a rota realizada pelo carro (dados mix)
* fazer o comparativo entre realizada e programada
* enviar email com os carros ofensores
* Todos os dados estão no banco postgres.

## Listando carro, motoristas e linhas que rodaram no dia

In [430]:
df_filtered = df_schedule[df_schedule['data'] == pd.Timestamp('2024-04-15')]

In [431]:
# Agrupar por 'carro' e agregar linhas e nomes únicos
df_report = df_filtered.groupby(['data','carro','linha']).agg({
    're': lambda x: x.unique(),
    'nome': lambda x: x.unique()
}).reset_index()


In [432]:
df_report

,data,carro,linha,re,nome
0,2024-04-15,1201,117A,"[000007200, 000018995]","[LINDOMAR ANTONIO GARCIA, RICARDO ALEXANDRE CH..."
1,2024-04-15,20017,020A,[000019061],[EDUARDO ROCHA DOS SANTOS]
2,2024-04-15,20017,032A,[000014901],[DERLEY RICARDO DA SILVA]
3,2024-04-15,20017,039A,[000014901],[DERLEY RICARDO DA SILVA]
4,2024-04-15,20017,186A,[000019061],[EDUARDO ROCHA DOS SANTOS]
...,...,...,...,...,...
1378,2024-04-15,20805,025C,[000017814],[EDIMAR RODRIGUES ROSA]
1379,2024-04-15,20998,026A,[000018886],[EZEQUIEL DA SILVA HUTIM]
1380,2024-04-15,20998,026B,[000018886],[EZEQUIEL DA SILVA HUTIM]
1381,2024-04-15,20998,027A,[000017239],[RONES SOARES DOS SANTOS]


## Adicionando as rotas ociosas ao dataframe

Adicionando ao df_report somente os campos importantes contidos no df_trip_idle

In [433]:
df_filtered = df_trip_idle[['id', 'num_linha', 'descricao', 'direcao', 'pontos_geolocalizacao']].copy()

In [434]:
df_report = pd.merge(df_report, df_filtered, left_on='linha',right_on='num_linha', how='left')

In [435]:
df_report.sample(n=5)

,data,carro,linha,re,nome,id,num_linha,descricao,direcao,pontos_geolocalizacao
2374,2024-04-15,20513,018B,[000018245],[JOSE ADRIANO RESENDE DE PAULA],53.0,018B,T. ARAGUAIA / PRA?A CIVICA - VIA BR-153,IDA,"[{'lat': -16.76356166845891, 'lng': -49.270145..."
2418,2024-04-15,20519,107A,[000017658],[GREGORIO JOSE DE SOUZA],91.0,107A,T. VL. BRASILIA / T. CRUZEIRO / VL. ALZIRA,IDA,"[{'lat': -16.76347333006561, 'lng': -49.270256..."
1592,2024-04-15,20425,519B,[000013969],[EDILSON LOPES PEREIRA],172.0,519B,T. VEIGA JD. / NOVA CIDADE / INDEP. MANSOES,IDA,"[{'lat': -16.76350454639546, 'lng': -49.270226..."
1036,2024-04-15,20320,003H,[000018347],[NELIO BOTOSSO],12.0,003H,PCA CIVICA / AVENIDA ITALIA / T. MARANATA,VOLTA,"[{'lat': -16.8113258583215, 'lng': -49.3626842..."
2685,2024-04-15,20545,011A,"[000018030, 000018264]","[DORVALINO XAVIER DOS ANJOS, DURVAL MARTINS RO...",41.0,011A,T. VL. BRASILIA / T. PRACA,VOLTA,"[{'lat': -16.74277825240883, 'lng': -49.255227..."


## Verificação de linhas que não possui cadastro no sianet de linhas ociosas

In [436]:
df_report.isna().sum()

data                     0
carro                    0
linha                    0
re                       0
nome                     0
id                       6
num_linha                6
descricao                6
direcao                  6
pontos_geolocalizacao    6
dtype: int64

In [437]:
rows_with_nan = df_report[df_report.isna().any(axis=1)] 

In [438]:
rows_with_nan

,data,carro,linha,re,nome,id,num_linha,descricao,direcao,pontos_geolocalizacao
0,2024-04-15,1201,117A,"[000007200, 000018995]","[LINDOMAR ANTONIO GARCIA, RICARDO ALEXANDRE CH...",NaN,NaN,NaN,NaN,NaN
659,2024-04-15,20080,362C,[000014077],[BELARMINO SANTANA NOGUEIRA],NaN,NaN,NaN,NaN,NaN
812,2024-04-15,20223,608B,[000019539],[SILVANIR DERMOND DIAS],NaN,NaN,NaN,NaN,NaN
939,2024-04-15,20303,608A,[000019464],[JOSE CLAUDIO DE PAULA],NaN,NaN,NaN,NaN,NaN
940,2024-04-15,20303,608B,[000019567],[JOSE NILTON FERREIRA DE SOUZA],NaN,NaN,NaN,NaN,NaN
1249,2024-04-15,20351,608A,[000019636],[JASON LOPES DE ARAUJO JUNIOR],NaN,NaN,NaN,NaN,NaN


## Adicionando as rotas das linhas

In [439]:
df_planned_trip

,name,coordinates,Código,identificacao,Descrição,Sentido,sublinha,num_linha
0,110-1i - T. PE. PELAGIO / T. SENADOR CANEDO - IDA,"-49.327230289,-16.659400785,0 -49.327226585000...",110,1i,T. PE. PELAGIO / T. SENADOR CANEDO,IDA,A,110A
1,110-1v - T. PE. PELAGIO / T. SENADOR CANEDO - ...,"-49.102709768,-16.696634942,0 -49.102510625,-1...",110,1v,T. PE. PELAGIO / T. SENADOR CANEDO,VOLTA,A,110A
2,111-1i - T. SEN. CANEDO / T. BIBLIA - IDA,"-49.102279636,-16.697228284,0 -49.102444883,-1...",111,1i,T. SEN. CANEDO / T. BIBLIA,IDA,A,111A
3,111-1v - T. SEN. CANEDO / T. BIBLIA - VOLTA,"-49.235356,-16.671601,0 -49.23670707099999,-16...",111,1v,T. SEN. CANEDO / T. BIBLIA,VOLTA,A,111A
4,112-1i - T. TRINDADE / T. BIBLIA - IDA,"-49.489861817,-16.651018366,0 -49.490032225,-1...",112,1i,T. TRINDADE / T. BIBLIA,IDA,A,112A
...,...,...,...,...,...,...,...,...
922,105-2v - T. PRACA A / CEMITERIO PARQUE - VOLTA,"-49.279291828,-16.641798643,0 -49.279555,-16.6...",105,2v,T. PRACA A / CEMITERIO PARQUE,VOLTA,B,105B
923,990-1i - T. BIBLIA / PRACA CIVICA / OSCAR NIEM...,"-49.235848,-16.671523,0 -49.236621539,-16.6715...",990,1i,T. BIBLIA / PRACA CIVICA / OSCAR NIEMEYER,IDA,A,990A
924,990-1v - T. BIBLIA / PRACA CIVICA / OSCAR NIEM...,"-49.226386406,-16.711469976,0 -49.225732416,-1...",990,1v,T. BIBLIA / PRACA CIVICA / OSCAR NIEMEYER,VOLTA,A,990A
925,991-1i - T. ISIDORIA / FLAMBOYANT / OSCAR NIEM...,"-49.252695,-16.713954,0 -49.25261492,-16.71358...",991,1i,T. ISIDORIA / FLAMBOYANT / OSCAR NIEMEYER,IDA,A,991A


In [440]:
df_filtered = df_planned_trip[['coordinates', 'Sentido','num_linha']]

In [441]:
df_report = pd.merge(df_report, df_filtered, left_on=['num_linha','direcao'],right_on=['num_linha','Sentido'], how='left')

## Adicionando a rota realizada

In [442]:
df_trip_made[df_trip_made['execution_date']=='2024-04-15']

,execution_date,registration,description,coordinates
476,2024-04-15,SCQ3A44,20805,"[[-49.301633, -16.705014], [-49.301468, -16.70..."
477,2024-04-15,NKC3574,20238,"[[-49.301015, -16.704094], [-49.301215, -16.70..."
478,2024-04-15,ONU6399,20018,"[[-49.301646, -16.704247], [-49.30138, -16.703..."
479,2024-04-15,NJY3685,20360,"[[-49.301201, -16.703086], [-49.301153, -16.70..."
480,2024-04-15,NLB6B14,20362,"[[-49.301187, -16.702887], [-49.301255, -16.70..."
481,2024-04-15,ONQ9479,20021,"[[-49.301364, -16.704743]]"
482,2024-04-15,NKE9854,20224,"[[-49.301699, -16.704075], [-49.301809, -16.70..."
483,2024-04-15,PQA2792,20048,"[[-49.362301, -16.81122], [-49.362246, -16.811..."
484,2024-04-15,ONW2749,20019,"[[-49.301651, -16.704151], [-49.301638, -16.70..."
485,2024-04-15,NLC7694,20357,"[[-49.301395, -16.702537], [-49.301461, -16.70..."


In [443]:
df_filtered = df_trip_made[['execution_date','description','coordinates']]

In [444]:
df_report['carro'] = df_report['carro'].astype('Int64')

In [445]:
df_report = pd.merge(df_report, df_filtered, left_on=['data', 'carro'], right_on=['execution_date', 'description'], how='left')


In [448]:
df_report.rename(columns={'pontos_geolocalizacao':'idle_route',
                          'coordinates_x': 'schedule_route',
                          'coordinates_y': 'made_route',})

,data,carro,linha,re,nome,id,num_linha,descricao,direcao,idle_route,schedule_route,Sentido,execution_date,description,made_route
0,2024-04-15,1201,117A,"[000007200, 000018995]","[LINDOMAR ANTONIO GARCIA, RICARDO ALEXANDRE CH...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,<NA>,NaN
1,2024-04-15,20017,020A,[000019061],[EDUARDO ROCHA DOS SANTOS],55.0,020A,T. GARAVELO / T. DA BIBLIA - VIA T. ISIDORIA,IDA,"[{'lat': -16.70507873620496, 'lng': -49.300697...","-49.349216,-16.765821,0 -49.348733,-16.765202,...",IDA,NaT,<NA>,NaN
2,2024-04-15,20017,020A,[000019061],[EDUARDO ROCHA DOS SANTOS],55.0,020A,T. GARAVELO / T. DA BIBLIA - VIA T. ISIDORIA,VOLTA,"[{'lat': -16.76513425403492, 'lng': -49.348903...","-49.236824038,-16.671738986,0 -49.236321,-16.6...",VOLTA,NaT,<NA>,NaN
3,2024-04-15,20017,032A,[000014901],[DERLEY RICARDO DA SILVA],80.0,032A,CIRCULAR / GARAVELO / VIA AV. DA PAZ,IDA,"[{'lat': -16.70507873620496, 'lng': -49.300697...","-49.349373,-16.765794,0 -49.349478519,-16.7659...",IDA,NaT,<NA>,NaN
4,2024-04-15,20017,032A,[000014901],[DERLEY RICARDO DA SILVA],80.0,032A,CIRCULAR / GARAVELO / VIA AV. DA PAZ,VOLTA,"[{'lat': -16.76513425403492, 'lng': -49.348903...","-49.327085,-16.774644,0 -49.326135034,-16.7741...",VOLTA,NaT,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,2024-04-15,20998,026B,[000018886],[EZEQUIEL DA SILVA HUTIM],70.0,026B,T. BANDEIRAS / PRACA DELMIRO,VOLTA,"[{'lat': -16.70948506823616, 'lng': -49.309943...",NaN,NaN,NaT,<NA>,NaN
3032,2024-04-15,20998,027A,[000017239],[RONES SOARES DOS SANTOS],72.0,027A,T. BANDEIRAS / T. DA BIBLIA - VIA LUIS DE MATOS,IDA,"[{'lat': -16.70513240150447, 'lng': -49.300648...","-49.310268,-16.710338,0 -49.31029029699999,-16...",IDA,NaT,<NA>,NaN
3033,2024-04-15,20998,027A,[000017239],[RONES SOARES DOS SANTOS],72.0,027A,T. BANDEIRAS / T. DA BIBLIA - VIA LUIS DE MATOS,VOLTA,"[{'lat': -16.70948506823616, 'lng': -49.309943...","-49.236824038,-16.671738986,0 -49.235273,-16.6...",VOLTA,NaT,<NA>,NaN
3034,2024-04-15,20998,028A,[000017239],[RONES SOARES DOS SANTOS],75.0,028A,T. BANDEIRAS / T. DA BIBLIA - VIA T-09,IDA,"[{'lat': -16.70513240150447, 'lng': -49.300648...","-49.31026,-16.710486,0 -49.31029386900001,-16....",IDA,NaT,<NA>,NaN


In [450]:
# Verifique as primeiras linhas do DataFrame resultante para confirmar o sucesso do merge
print(df_report.head())

# Verificar se há valores NaN que indicariam falhas na junção
print(df_report.isnull().sum())


        data  carro linha                      re  \
0 2024-04-15   1201  117A  [000007200, 000018995]   
1 2024-04-15  20017  020A             [000019061]   
2 2024-04-15  20017  020A             [000019061]   
3 2024-04-15  20017  032A             [000014901]   
4 2024-04-15  20017  032A             [000014901]   

                                                nome    id num_linha  \
0  [LINDOMAR ANTONIO GARCIA, RICARDO ALEXANDRE CH...   NaN       NaN   
1                         [EDUARDO ROCHA DOS SANTOS]  55.0      020A   
2                         [EDUARDO ROCHA DOS SANTOS]  55.0      020A   
3                          [DERLEY RICARDO DA SILVA]  80.0      032A   
4                          [DERLEY RICARDO DA SILVA]  80.0      032A   

                                      descricao direcao  \
0                                           NaN     NaN   
1  T. GARAVELO / T. DA BIBLIA - VIA T. ISIDORIA     IDA   
2  T. GARAVELO / T. DA BIBLIA - VIA T. ISIDORIA   VOLTA   
3          C